<a href="https://colab.research.google.com/github/1337DS/DrunkFaceRecognition/blob/main/drunk_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade=cv2.CascadeClassifier("haarcascade_eye.xml")
import mediapipe as mp
import numpy as np
import imutils
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
from PIL import Image

import pandas as pd
import dlib
import cv2
import os


In [2]:
def check_for_face(img_path):
    # Load the face detector
    detector = dlib.get_frontal_face_detector()
    # Read the input image
    img = cv2.imread(img_path)
    # Convert into grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imshow('Gray', gray)
    # Terminate the process
    #cv2.waitKey(0)
    #cv2.destroyWindow('Gray')
    #cv2.waitKey(1)
    # Detect faces
    detector = dlib.get_frontal_face_detector()
    face = detector(gray, 0)
    #print(type(face))
    if len(face) != 0:
        face_detected = True
    else:
        face_detected = False
    #cv2.imshow('Face', img)
    # Terminate the process
    #cv2.waitKey(0)
    #cv2.destroyWindow('Face')
    #cv2.waitKey(1)
    return face_detected

In [4]:

def get_facial_landmarks(img_path):
    # INITIALIZING OBJECTS
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_face_mesh = mp.solutions.face_mesh
    
    cap = cv2.imread(img_path)
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
        image = cap

        # Flip the image horizontally and convert the color space from BGR to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        # To improve performance
        image.flags.writeable = False
        
        # Detect the face landmarks
        results = face_mesh.process(image)

        # To improve performance
        image.flags.writeable = True

        # Convert back to the BGR color space
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw the face mesh annotations on the image.
        face_landmarks = results.multi_face_landmarks[0].landmark
        #print(face_landmarks)
        #print(type(face_landmarks))
        data = []
        for i in range(len(face_landmarks)):
            #print(face_landmarks[i])
            landmark = face_landmarks[i]
            x = landmark.x
            y = landmark.y
            z = landmark.z
            
            data.append(x)
            data.append(y)
            data.append(z)
            
            
            # draw the landmarks on as img
            #mp_drawing.draw_landmarks(
            #    image=image,
            #    landmark_list=face_landmark,
            #    connections=mp_face_mesh.FACEMESH_TESSELATION,
            #    landmark_drawing_spec=None,
            #    connection_drawing_spec=mp_drawing_styles
            #    .get_default_face_mesh_tesselation_style())
            # Display the image
            #cv2.imshow('MediaPipe FaceMesh', image)
            # Terminate the process
            #cv2.waitKey(0)
            #cv2.destroyWindow('MediaPipe FaceMesh')
            #cv2.waitKey(1)
        
        #print(data)
        return data
        
            
            
            
            


path = '/Users/yannikhubrich/Documents/Studium/6Semester/DrunkFaceRecognition/Data/raw/alligned/d4a7a2_1b7fef8b5225436f923db06a76c52e4a.jpg_drei_glaeser.jpg'
data = get_facial_landmarks(path)

In [5]:
from os import listdir
from os.path import isfile, join
path = '/Users/yannikhubrich/Documents/Studium/6Semester/DrunkFaceRecognition/Data/raw/alligned'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

col = ['how_many_glasses_drunk']
for i in range(468):
    i = i+1
    col.append(f'flm {i} x')
    col.append(f'flm {i} y')
    col.append(f'flm {i} z')

#print(len(col)) 

df = pd.DataFrame(columns=col)

for i in onlyfiles:
    img_path = f'/Users/yannikhubrich/Documents/Studium/6Semester/DrunkFaceRecognition/Data/raw/alligned/{i}'
    #print(i)
    if 'null_glas' in i:
        how_many_glasses_drunk = 0
    elif 'ein_glas' in i:
        how_many_glasses_drunk = 1
    elif 'zwei_glaeser' in i:
        how_many_glasses_drunk = 2
    elif 'drei_glaeser' in i:
        how_many_glasses_drunk = 3
    
    data = get_facial_landmarks(img_path)
    data.insert(0, how_many_glasses_drunk)
    df.loc[len(df.index)] = data
    
df

,how_many_glasses_drunk,flm 1 x,flm 1 y,flm 1 z,flm 2 x,flm 2 y,flm 2 z,flm 3 x,flm 3 y,flm 3 z,...,flm 465 z,flm 466 x,flm 466 y,flm 466 z,flm 467 x,flm 467 y,flm 467 z,flm 468 x,flm 468 y,flm 468 z
0,0.0,0.473866,0.402868,-0.025370,0.471838,0.363766,-0.062777,0.471644,0.374617,-0.029541,...,-0.006866,0.483343,0.285906,-0.015141,0.548663,0.264883,0.009346,0.553632,0.260789,0.009512
1,3.0,0.529035,0.479753,-0.045692,0.527123,0.423836,-0.089461,0.528313,0.441239,-0.047993,...,0.001182,0.558589,0.339630,-0.011470,0.647178,0.331790,0.016733,0.655496,0.328523,0.016815
2,0.0,0.488613,0.465995,-0.040977,0.491965,0.406201,-0.085306,0.490271,0.423701,-0.042970,...,-0.002360,0.518228,0.309380,-0.013877,0.605243,0.298787,0.017503,0.612576,0.294229,0.017922
3,0.0,0.543289,0.507316,-0.039361,0.534558,0.451950,-0.087020,0.538568,0.466730,-0.043215,...,-0.007188,0.559555,0.348770,-0.018709,0.651529,0.325647,0.007015,0.658861,0.320907,0.006656
4,2.0,0.460799,0.484583,-0.046623,0.450390,0.428057,-0.085037,0.454610,0.447449,-0.046419,...,0.001234,0.468582,0.338643,-0.009183,0.555283,0.321713,0.017588,0.563433,0.313085,0.018635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,1.0,0.530306,0.464028,-0.024733,0.525677,0.433416,-0.067727,0.527758,0.442051,-0.033420,...,-0.013429,0.543823,0.359391,-0.021670,0.611415,0.343561,-0.002721,0.617543,0.338430,-0.002690
193,2.0,0.484086,0.480425,-0.040968,0.476703,0.443716,-0.083251,0.480136,0.456226,-0.045193,...,-0.004153,0.497965,0.354442,-0.014747,0.579973,0.332144,0.008531,0.588153,0.326109,0.008526
194,1.0,0.463084,0.451829,-0.048814,0.471539,0.402548,-0.077351,0.467221,0.420056,-0.045969,...,0.011141,0.495300,0.341182,0.000616,0.565928,0.345327,0.032542,0.574313,0.340227,0.034021
195,1.0,0.481256,0.463324,-0.036111,0.477403,0.410858,-0.080956,0.479005,0.426323,-0.040194,...,-0.006648,0.500427,0.327543,-0.017693,0.581755,0.314175,0.007886,0.589080,0.311543,0.007732


In [6]:
from PIL import Image
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from imgaug import augmenters as iaa
import random

def get_image(image_path):
    img = cv2.imread(image_path)
    img_file_name = image_path.split('/')[-1]
    print(img_file_name)
    return img, img_file_name

def change_brightness(cv2_img, file_name):
    hsv = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2HSV)
    value = random.randint(20, 60) #whatever value you want to add
    
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,value)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

    #cv2.imshow('test', img)
    #cv2.waitKey(0)
    #cv2.destroyWindow('test')
    #cv2.waitKey(1)
    
    file_name = 'brighness_' + file_name
    
    path_to_safe = '/Users/yannikhubrich/Documents/Studium/6Semester/DrunkFaceRecognition/Data/final' + file_name
    cv2.imwrite(path_to_safe, img)
    
    return img
    
img, img_file_name = get_image('/Users/yannikhubrich/Documents/Studium/6Semester/DrunkFaceRecognition/Data/raw/alligned/d4a7a2_1b7fef8b5225436f923db06a76c52e4a.jpg_drei_glaeser.jpg')
change_brightness(img, img_file_name)

d4a7a2_1b7fef8b5225436f923db06a76c52e4a.jpg_drei_glaeser.jpg
